<a href="https://colab.research.google.com/github/ChristinaKou/Congitive-/blob/Classifier/CognvitivePlus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Christina Koutsoumpa** -  Classification of small images







# **1. Initialisation and function definitions**

Import modules

In [0]:
import os
import numpy as np
import matplotlib.pylab as plt 
import tarfile
from random import randrange


Paths

In [0]:
# Project directory
workingDir = '/Cognitive'
# Make a project folder if not existent
if not os.path.exists(workingDir):
    os.makedirs(workingDir)
os.chdir(workingDir) # Cd to this foder 
print("The current workig directory is",os.getcwd()[1:] )

Functions

In [0]:
# List non hidden files in current folder
def nonHiddenfiles():
  fileList = [n_file for n_file in os.listdir() if not n_file.startswith('.')] 
  return fileList

# Unpickle file
def unpickle(file):
  import pickle
  with open(file, 'rb') as fo:
    dict = pickle.load(fo, encoding='bytes')
  return dict


# **2. Data Loading and preprocessing**



## 2.1 Download and decompress dataset

In [0]:
# Download dataset if not already downloaded
fileList = nonHiddenfiles()
if len(fileList)==0: # Check for non hidden files
  # Donwload 
  print("Downloading data ..")
  !wget -q --show-progress https://www.cs.toronto.edu/%7Ekriz/cifar-100-python.tar.gz
  fileList = nonHiddenfiles()
  print("Dataset", fileList[-1],  "downloaded" )
else:
  # Do not download
  print("Dataset", fileList[-1], "already downloaded" )
  fileList = nonHiddenfiles()

In [0]:
# Decompress dataset
fileList = nonHiddenfiles()
datasetFile = fileList[-1]
if datasetFile.endswith("tar.gz"):
  print("Decompressing dataset..")    
  tar = tarfile.open(datasetFile, "r:gz")    
  tar.extractall()
  tar.close()
  print("..into", tar.name)
  os.remove(tar.name) # delete compressed file
  fileList.remove(datasetFile)
else:
  print("Dataset uncompressed") 


## 2.2 Explore dataset

In [0]:
# Look into the folder
os.chdir(tar.members[0].name) 
# List non hidden content
contentList = nonHiddenfiles()
print(*contentList, sep = "\n")
%cd ..

In [0]:
# Unpickle data 
os.chdir(tar.members[0].name)
test = unpickle(contentList[0])
meta = unpickle(contentList[1])
train = unpickle(contentList[3])
print(type(test), type(meta), type(train))
%cd ..

### Table of dataset summary 

In [0]:
print("\tLABEL","\t\t\t","LENGTH\t\t", "VALUE")

# expore meta dataset
print("META")
for i in list(range(0,len(meta))):
  print(list(meta.keys())[i],"\t\t",len(list(test.values())[i]),"\t\t",list(meta.values())[i]), print('\n')

# expore train dataset
print("TRAIN")
for i in list(range(0,len(train))):
  print(list(train.keys())[i],"      \t\t",len(list(train.values())[i]),"\t\t",list(train.values())[i]), print('\n')

# expore test dataset
print("TEST")
for i in list(range(0,len(test))):
  print(list(test.keys())[i],":          \t",len(list(test.values())[i]),"\t\t", list(test.values())[i]), print('\n')


### Image examples

In [0]:
# type and shape of image data
print(type(train[b'data']),train[b'data'].shape)


In [0]:
# reshape dataset
trainImages = train[b'data'].reshape(50000,3,32,32).transpose(2,3,1,0);
trainImages.shape

In [0]:
# Show random image examples with their filename, coarse category and fine category
%matplotlib inline
thisImage=randrange(len(train[b'filenames']));
plt.imshow(trainImages[:,:,:,thisImage].squeeze());
plt.title([train[b'filenames'][thisImage], meta[b'fine_label_names'][train[b'fine_labels'][thisImage]],meta[b'coarse_label_names'][train[b'coarse_labels'][thisImage]]] );


# **3. Classifier training**


Import modules

In [0]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense


Initialisation

In [0]:
# Clear any logs from previous runs
!rm -rf ./logs/

# 3.1. Model

In [0]:
classifier = Sequential()
# First convolutional layer 
classifier.add(Convolution2D(32, (3, 3), input_shape = (32, 32, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Second convolutional layer 
classifier.add(Convolution2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Flattening
classifier.add(Flatten())

# Full connection
classifier.add(Dense(output_dim = 128, activation = 'relu'))
classifier.add(Dense(output_dim = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Print topology
classifier.summary(80)

* Layers: 2 convolutional and max-pooling layers, 1 flatten layer, 2 fully connected layers with ReLU and sigmoid
* Activation function : ReLU
* Loss function : cross entropy
* Optimiser: Adam

In [0]:
%matplotlib notebook

from keras.utils import plot_model
plot_model(classifier, to_file='model.png')

#from IPython.display import Image 
#Image('model.png')

In [0]:
%load_ext tensorboard
%tensorboard --logdir logs

In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   rotation_range=15,
                                   vertical_flip=True,
                                   fill_mode='reflect',
                                   data_format='channels_last',
                                   brightness_range=[0.5, 1.5],
                                   featurewise_center=True,
                                   featurewise_std_normalization=True)




In [0]:
test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/validation',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Activation function ReLU, leaky ReLU, Swish

Multi-Class Cross-Entropy Loss, Sparse Multiclass Cross-Entropy Loss, Kullback Leibler Divergence Loss

adam, sgd



# Citations
1. https://www.cs.toronto.edu/~kriz/cifar.html (dataset)
2. https://www.tensorflow.org/tensorboard/graphs 